# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Access the PRICE_DATA environment variable
PRICE_DATA = os.getenv('PRICE_DATA')
print(PRICE_DATA)

../../05_src/data/prices/


In [45]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [61]:
PRICE_DATA

'../../05_src/data/prices/'

In [49]:
import os
from glob import glob
import glob as glob_module

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files  = glob(os.path.join(PRICE_DATA, '*/*/*.parquet'))
   

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [57]:
pip install --upgrade dask

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 0.4.1 requires dask==2024.2.0, but you have dask 2024.7.0 which is incompatible.
distributed 2024.2.0 requires dask==2024.2.0, but you have dask 2024.7.0 which is incompatible.


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------- -------------------------------- 0.2/1.2 MB 4.8 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.2 MB 5.6 MB/s eta 0:00:01
   ------------------------ --------------- 0.8/1.2 MB 5.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.0/1.2 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2024.2.0
    Uninstalling dask-2024.2.0:
      Successfully uninstalled dask-2024.2.0


In [58]:
# Write your code below.
from dotenv import load_dotenv
import os
from glob import glob
import dask.dataframe as dd
# Read all parquet files into a Dask DataFrame
ddf = dd.read_parquet(parquet_files)

# Display the columns to confirm the structure
print(ddf.columns)

# Ensure 'Date' is in datetime format if not already
ddf['Date'] = dd.to_datetime(ddf['Date'])

# Sort by sector, subsector, and Date to ensure the correct order for lag calculations
ddf = ddf.sort_values(by=['sector', 'subsector', 'Date'])

# Add lagged features for Close
ddf['Close_lag'] = ddf.groupby(['sector', 'subsector'])['Close'].shift(1)

# Calculate returns based on Close
ddf['returns'] = (ddf['Close'] / ddf['Close_lag']) - 1

# Calculate the day's high-low range
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Persist the result to avoid recomputing
dd_feat = ddf.persist()


# Display the first few rows of the resulting DataFrame to verify the transformations
print(dd_feat.head())

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


C:\Users\yezhu\AppData\Local\Temp\ipykernel_53424\1117178657.py:19: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Close_lag'] = ddf.groupby(['sector', 'subsector'])['Close'].shift(1)


AttributeError: 'AssignAlign' object has no attribute 'keys'

This often means that you are attempting to use an unsupported API function. Current API coverage is documented here: https://github.com/dask-contrib/dask-expr/blob/main/README.md#api-coverage.

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [59]:
# Write your code below.

# Convert the Dask DataFrame to a Pandas DataFrame
pd_df = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days
pd_df['rolling_avg_return'] = pd_df.groupby(['sector', 'subsector'])['returns'].transform(lambda x: x.rolling(10).mean())

# Display the first few rows of the resulting DataFrame to verify the transformations
print(pd_df.head())


NameError: name 'dd_feat' is not defined

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)
It's not necessary to convert to Pandas to calculate the moving average return. yes,  Dask, you can avoid loading the entire dataset into memory.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.